In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np

In [ ]:
res = pd.read_csv( snakemake.input.results )
res[["reads", "trial"]] = res["sequence_id"].str.split( ".", expand=True )[[1,2]]
res["actual"] = "T15"
res["correct"] = res["actual"]==res["lineage"]
res["depth"] = res["depth"].str.rstrip('%').astype('float') / 100
res.loc[~res["correct"]&(res["confidence"]>0.5),"confidence"] = 1 - res["confidence"]

coverage = res.pivot( columns="reads", index="trial", values="depth")

res["reads"] = pd.to_numeric( res["reads"] )
res.head()

In [ ]:
summ = res.groupby( "reads" )["correct"].agg( ["count", "sum"] )
summ.columns = ["observations", "successes"]
summ = summ.reset_index()
summ["accuracy"] = summ["successes"] / summ["observations"]
summ[["accuracy_low", "accuracy_high"]] = summ.apply( lambda x: pd.Series( proportion_confint( x["successes"], x["observations"], alpha=0.05, method="jeffreys" ) ), axis=1 )
summ.head()

In [ ]:
fig, ax = plt.subplots( dpi=200, figsize=(7,4), ncols=2, sharex=True, gridspec_kw={"width_ratios" : (4,2)} )
ax[0].plot( summ["reads"], summ["accuracy"], color="black", linewidth=1, zorder=15 )
ax[0].fill_between( summ["reads"], summ["accuracy_low"], summ["accuracy_high"], color="black", linewidth=0, alpha=0.2, zorder=10 )
ax[0].set_xscale( "log" )
ax[0].yaxis.set_major_formatter( mticker.PercentFormatter(1) )
ax[0].set_yticks( np.arange( 0, 1, 0.05 ), minor=True )

ax[0].grid( which="major", axis="both", linewidth=1, color="#F1F1F1", zorder=1 )
ax[0].grid( which="minor", axis="both", linewidth=0.5, color="#F1F1F1", zorder=1 )

ax[0].set_ylim(0, 1.01)
ax[0].set_xlim(50, 5000000)

ax[0].set_ylabel( "Accuracy [%]", fontweight="bold")
ax[0].set_xlabel( "Cholera reads", fontweight="bold" )


points = ax[1].scatter( res["reads"], res["depth"], zorder=100, c="#009E73", s=10 )
points.set_clip_on(False)
ax[1].yaxis.set_major_formatter( mticker.PercentFormatter(1) )
ax[1].set_yticks( np.arange( 0, 1, 0.05 ), minor=True )

ax[1].set_xlabel( "Cholera reads", fontweight="bold")
ax[1].set_ylabel( "Genome coverage [%]", fontweight="bold")

ax[1].grid( which="major", axis="both", linewidth=1, color="#F1F1F1", zorder=1 )
ax[1].grid( which="minor", axis="both", linewidth=0.5, color="#F1F1F1", zorder=1 )

ax[1].set_ylim(0, 1.01)

plt.tight_layout()
plt.savefig( snakemake.output.accuracy_coverage_plot )
plt.show()